# Training the Model

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train_final.csv",index_col=False)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,73,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
3,0,0,0,0,0,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
4,0,0,0,0,0,0,0,0,18,255,...,0,0,0,0,0,0,0,0,0,10


In [3]:
labels = df_train["784"]

In [4]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,73,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,18,255,...,0,0,0,0,0,0,0,0,0,0


In [5]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input,Dense
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [8]:
labels=np.array(labels)

In [9]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes= 13)

In [10]:
l=[]
for i in range(5946):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))

In [11]:
np.random.seed(7)

In [12]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape =(28, 28, 1), activation ='relu'))
model.add(MaxPooling2D(pool_size =(2, 2)))
model.add(Conv2D(15, (3, 3), activation ='relu'))
model.add(MaxPooling2D(pool_size =(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dense(50, activation ='relu'))
model.add(Dense(13, activation ='softmax'))

In [13]:
model.compile(loss ='categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [14]:
model.fit(np.array(l), cat, epochs = 25, batch_size = 200, shuffle = True, verbose = 1)

Epoch 1/25
30/30 [==============================] - 5s 131ms/step - loss: 8.4850 - accuracy: 0.2193
Epoch 2/25
30/30 [==============================] - 4s 126ms/step - loss: 1.4837 - accuracy: 0.5457
Epoch 3/25
30/30 [==============================] - 4s 128ms/step - loss: 0.8441 - accuracy: 0.7513
Epoch 4/25
30/30 [==============================] - 4s 142ms/step - loss: 0.5638 - accuracy: 0.8311
Epoch 5/25
30/30 [==============================] - 4s 128ms/step - loss: 0.4194 - accuracy: 0.8801
Epoch 6/25
30/30 [==============================] - 4s 145ms/step - loss: 0.3331 - accuracy: 0.8981
Epoch 7/25
30/30 [==============================] - 4s 137ms/step - loss: 0.2644 - accuracy: 0.9173
Epoch 8/25
30/30 [==============================] - 4s 128ms/step - loss: 0.2361 - accuracy: 0.9272
Epoch 9/25
30/30 [==============================] - 4s 120ms/step - loss: 0.2009 - accuracy: 0.9405
Epoch 10/25
30/30 [==============================] - 4s 126ms/step - loss: 0.1803 - accuracy: 0.9428

# Saving our Model

This helps to avoid the time to train again and again

In [65]:
import json
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")

# Calling the saved model

In [69]:
from keras.models import model_from_json
json_file = open('model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_final.h5")

# Testing the Model

In [19]:
import cv2
img=cv2.imread("C:/Users/Admin/Desktop/test/test3.jpeg",cv2.IMREAD_GRAYSCALE)
cv2.imshow("w",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
if img is not None:
    img= ~img
    _,thresh= cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    ctrs,_=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt=sorted(ctrs,key=lambda ctr:cv2.boundingRect(ctr)[0])
    w=int(28)
    h=int(28)
    train_data=[]
    rects=[]
    for c in cnt:
        x,y,w,h = cv2.boundingRect(c)
        rect=[x,y,w,h]
        rects.append(rect)
    print("rects",rects)
    bool_rects=[]
    for r in rects:
        l=[]
        for rec in rects:
            flag = 0
            if rec != r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)
            if rec==r:
                l.append(0)
        bool_rects.append(l)
    print("bools",bool_rects)
    dump_rect = []
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rects[i][j] == 1:
                area1 = rects[i][2]*rects[i][3]
                area2 = rects[j][2]*rects[j][3]
                if area1 == min(area1,area2):
                    dump_rect.append(rects[i])  
    print("dump_rects",dump_rect)
    final_rects=[i for i in rects if i not in dump_rect]
    print("final rects",final_rects)
    for r in final_rects:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop = thresh[y:y+h+10,x:x+w+10]
        im_resize = cv2.resize(im_crop,(28,28))
        cv2.imshow("work",im_resize)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        train_data.append(im_resize)
         

rects [[116, 349, 263, 26], [145, 218, 66, 134], [181, 360, 2, 1], [186, 229, 2, 3], [190, 358, 7, 3], [202, 357, 3, 3], [206, 253, 3, 5], [208, 357, 3, 1], [215, 356, 5, 2], [274, 352, 1, 2], [310, 353, 2, 1], [448, 214, 147, 172], [507, 268, 12, 5], [696, 209, 173, 154]]
bools [[0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
dump_rects [[116, 349, 263, 26], [181, 360, 2, 1], [181, 360, 2, 1], [181, 360, 2, 1], [186, 229, 2, 3]

In [25]:
s=""
for i in range(len(train_data)):
    train_data[i]= np.array(train_data[i])
    train_data[i] = train_data[i].reshape(1,28,28,1)
    res = model.predict(train_data[i])
    result= np.argmax(res)
    if (result == 10):
        s=s+"-"
    if (result == 11):
        s=s+"+"
    if (result == 0):
        s=s+"0"
    if (result == 1):
        s=s+"1"
    if (result== 2):
        s=s+"2"
    if (result== 3):
        s=s+"3"
    if (result == 4):
        s=s+"4"
    if (result == 5):
        s=s+"5"
    if (result == 6):
        s=s+"6"
    if (result == 7):
        s=s+"7"
    if (result == 8):
        s=s+"8"
    if (result == 9):
        s=s+"9"
    if (result == 12):
        s=s+"*"
print(s)

1/1 [==============================] - 0s 27ms/step
2+3


In [26]:
eval(s)

5